In [218]:
import praw
import pandas as pd
import numpy as np
from praw.models import MoreComments
import time
import numpy as np

In [33]:
reddit = praw.Reddit(client_id='my client id',
                     client_secret='my client secret',
                     user_agent='my user agent',
                     username='my username',
                     password='my password')

In [190]:
def get_commentiers(submission):
    temp=[]
    for top_level_comment in submission.comments:
        if isinstance(top_level_comment, MoreComments):
            continue
        temp.append(top_level_comment.author)
    return temp

In [191]:
def get_comments_subs(redditor):
    temp=[]
    if redditor!=None:
        for comment in redditor.comments.new(limit=500):
            temp.append(comment.subreddit.display_name)
    return temp

In [198]:
def thread2subreddits(ID):
    submission = reddit.submission(ID)
    commentiers=get_commentiers(submission)
    index=[]
    for commentier in commentiers:
        if commentier!=None:
            if commentier.name in all_users:
                commentiers.remove(commentier)
            else:
                index.append(commentier.name)
                all_users.append(commentier.name)
    df=pd.DataFrame(index=index)
    i=0
    for commentier in commentiers:
        if commentier!=None:
            if commentier.name in index:
                print('Redditor no. '+str(i+1)+' out of '+str(len(commentiers)))
                subs=set(get_comments_subs(commentier))
                for sub in subs:
                    if sub in df.columns:
                        df[sub].loc[commentier.name]=df[sub].loc[commentier.name]+1
                    else:
                        df[sub]=0
                        df[sub].loc[commentier.name]=1
        i+=1
    return df

In [193]:
def thread2subredditsDic(ID):
    subs={}
    submission = reddit.submission(ID)
    commentiers=get_commentiers(submission)
    for commentier in commentiers:
        subreddits=set(get_comments_subs(commentier))
        for sub in subreddits:
            if sub in subs.keys():
                subs[sub]=subs[sub]+1
            else:
                subs[sub]=1
    return subs

In [194]:
def get_new(name):
    temp=[]
    for a in reddit.subreddit('dankmemes').new():
        if a in all_threads:
            break
        temp.append(a)
        all_threads.append(a)
    return temp

In [202]:
def run_full(subreddit,runs=60,wait=600):
    full_df=pd.DataFrame()
    i=0
    for run in range(runs):
        print('Run '+str(i+1)+' out of '+str(runs))
        threads=get_new(subreddit)
        j=0
        for thread in threads:
            print('Thread '+str(j+1)+' out of '+str(len(threads)))
            subs_df=thread2subreddits(thread)
            full_df=pd.concat([subs_df,full_df])
            j+=1
        print('Finished run, waiting '+str(wait)+'seconds')
        time.sleep(wait)
        i+=1
        if i%10==0:
            full_df2=full_df.fillna(0)
            total = full_df2.apply(np.sum)
            full_df2=full_df2.append(pd.DataFrame(total.values, index=total.keys()).T)
            as_list = full_df2.index.tolist()
            idx = as_list.index(0)
            as_list[idx] = 'Sum'
            full_df2.index = as_list
            full_df2.sort_values(by='Sum',ascending=False,axis=1)
            full2.to_csv(str(i)+'.csv')
    
    full_df2=full_df.fillna(0)
    total = full_df2.apply(np.sum)
    full_df2=full_df2.append(pd.DataFrame(total.values, index=total.keys()).T)
    as_list = full_df2.index.tolist()
    idx = as_list.index(0)
    as_list[idx] = 'Sum'
    full_df2.index = as_list
    full_df2.sort_values(by='Sum',ascending=False,axis=1)
    full2.to_csv('things.csv')
    return full_df2

In [302]:
all_threads=[]
all_subs=[]
all_users=[]
full=run_full('dankmemes',runs=300,wait=150) # first argument is subreddit name. wait decides seconds waited after each full run, set higher for less active subs. runs decides amount of runs, set higher for more data.